In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'quanta.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from apps.orders.models import Consultation, Commercial
from apps.organization.models import Organization
from django.db.models import Count, F, Func, Q, Sum, Value, Window


In [3]:
from apps.organization.models import Organization

In [4]:
organization = Organization.objects.first()

In [7]:
orders = Consultation.objects.filter(organization__in=organization.get_descendants(include_self=True))

In [25]:
facturation_report = orders.aggregate(
    count = Count("id"), 
    total_price= Sum("processing_price")
)

In [26]:
commercial_report = orders.values("commercial_id").annotate(
    count = Count("id"), 
    total_price= Sum("processing_price")
)

In [27]:
commercial_report

<DataFrameQuerySet [{'commercial_id': UUID('2b56b853-9430-4edb-8373-6db1ccb293b3'), 'count': 13, 'total_price': Decimal('256050')}]>

In [23]:
commercials = Commercial.objects.all()

In [33]:
commercial_kpi_reports = []
for commercial in commercials:
    report = [data for data in commercial_report if data["commercial_id"] == commercial.id]
    if report:
        commercial_kpi_reports.append({
            "organization": commercial.organization.name,
            "commercial": commercial.organization_user.user.username,
            "count": report[0]["count"],
            "total_price": report[0]["total_price"],
        })
    else:
        commercial_kpi_reports.append({
            "organization": commercial.organization.name,
            "commercial": commercial.organization_user.user.username,
            "count": 0,
            "total_price": 0
        })
        
print(commercial_kpi_reports)

[{'commercial': 'SuperUser', 'count': 13, 'total_price': Decimal('256050')}]


In [34]:
organization_report = orders.values("organization_id").annotate(
    count = Count("id"), 
    total_price= Sum("processing_price")
)

In [35]:
organizations = organization.get_descendants(include_self=True)

In [32]:
organization_kpi_reports = []
for organization in organizations:
    report = [data for data in organization_report if data["organization_id"] == organization.id]
    if report:
        organization_kpi_reports.append({
            "organization": organization.name,
            "count": report[0]["count"],
            "total_price": report[0]["total_price"],
        })
    else:
        organization_kpi_reports.append({
            "organization": organization.name,
            "count": 0,
            "total_price": 0
        })
        
print(organization_kpi_reports)

[{'organization': 'distrivite sarl', 'count': 13, 'total_price': Decimal('256050')}]
